In [46]:
from keras.layers import Activation,Convolution2D, Dropout,Conv2D,Dense
from keras.layers import AveragePooling2D,BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras.callbacks import ReduceLROnPlateau
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
import numpy as np
import scipy.misc
from imutils import paths
import cv2
import os
import random

In [47]:
def modelCNN():
    model = Sequential()
    model.add(Convolution2D(filters=64, kernel_size=(3,3), padding="same", activation="elu",
                           name='image_array',input_shape=(150,150,3)))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=64, kernel_size=(3,3), padding="same", activation="elu",
                           strides=(2,2)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.5))
    
    model.add(Convolution2D(filters=128, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=128, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Convolution2D(filters=256, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=256, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=256, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Convolution2D(filters=512, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=512, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=512, kernel_size=(3,3), padding="same", activation="elu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    
    model.add(layers.Flatten())
    model.add(Dropout(0.5))
    model.add(layers.Dense(4096,activation='elu'))
    #model.add(Dropout(0.5))
    model.add(layers.Dense(4096,activation='elu'))
    model.add(layers.Dense(2,activation='sigmoid'))
    
    return model

# Carregando Dados

In [48]:
data = []
labels = []

imagePaths = sorted(list(paths.list_images('/home/jailsonpereira/PAIC_CNN/versao_1')))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image,(150,150))
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    

data = np.array(data,dtype='float32')/255
labels = np.array(labels)
print(data.shape)

(223, 150, 150, 3)


# Pré-processando dados

In [49]:
le=LabelEncoder()
labels=le.fit_transform(labels)
labels=to_categorical(labels,num_classes=2)

In [50]:
X_train,X_val,Y_train,Y_val=train_test_split(data,labels,test_size=0.1)

# Intanciando arquitetura

In [51]:
model = modelCNN()

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 150, 150, 64)      1792      
_________________________________________________________________
batch_normalization_31 (Batc (None, 150, 150, 64)      256       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 75, 75, 64)        36928     
_________________________________________________________________
batch_normalization_32 (Batc (None, 75, 75, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 37, 37, 128)       73856     
__________

In [53]:
opt = optimizers.RMSprop(lr=0.01,rho=0.9,epsilon=1e-08,decay=0.0)

In [54]:
model.compile(opt,loss='binary_crossentropy',metrics=['accuracy'])

In [55]:
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [56]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,
        vertical_flip=False) 

datagen.fit(X_train)

In [57]:
history=model.fit_generator(datagen.flow(X_train,Y_train,batch_size=10),epochs=10,validation_data=(X_val,Y_val),
                          verbose=1,steps_per_epoch=X_train.shape[0]/10, callbacks=[learning_rate_reduction])

Epoch 1/10
20/20 [==============================] - 53s 3s/step - loss: 4.3309 - acc: 0.7075 - val_loss: 4.1818 - val_acc: 0.7391
Epoch 2/10
20/20 [==============================] - 49s 2s/step - loss: 4.4083 - acc: 0.7250 - val_loss: 4.1818 - val_acc: 0.7391
Epoch 3/10
20/20 [==============================] - 51s 3s/step - loss: 4.4083 - acc: 0.7250 - val_loss: 4.1818 - val_acc: 0.7391
Epoch 4/10
20/20 [==============================] - 49s 2s/step - loss: 4.4083 - acc: 0.7250 - val_loss: 4.1818 - val_acc: 0.7391

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
Epoch 5/10
20/20 [==============================] - 49s 2s/step - loss: 4.4083 - acc: 0.7250 - val_loss: 4.1818 - val_acc: 0.7391
Epoch 6/10
20/20 [==============================] - 49s 2s/step - loss: 4.4083 - acc: 0.7250 - val_loss: 4.1818 - val_acc: 0.7391
Epoch 7/10
20/20 [==============================] - 49s 2s/step - loss: 4.4083 - acc: 0.7250 - val_loss: 4.1818 - val_acc: 0.7391

Epoch 000